In [2]:
import numpy as np
import tensorflow as tf

tf.reset_default_graph()
sess = tf.InteractiveSession()

tf.__version__

/home/felipe/anaconda3/envs/deep-learning/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'1.4.1'

In [3]:
start_token = '\t'
end_token = '\n'
batch_size = 100
epochs = 100
latent_dim = 256
num_samples = 10000
data_path = 'data/fra.txt'

In [4]:
input_seqs = []
target_seqs = []
input_tokens = set()
target_tokens = set()

with open(data_path, 'r', encoding='utf-8') as file:
    lines = file.read().split('\n')
    
for line in lines[: min(num_samples, len(lines)-1)]:
    input_seq, target_seq = line.split('\t')
    
    target_seq = start_token + target_seq + end_token
    
    input_seqs.append(input_seq)
    target_seqs.append(target_seq)
    
    for token in input_seq:
        if token not in input_tokens:
            input_tokens.add(token)
            
    for token in target_seq:
        if token not in target_tokens:
            target_tokens.add(token)
            
input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))

num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

max_encoder_seq_length = max([len(seq) for seq in input_seqs])
max_decoder_seq_length = max([len(seq) for seq in target_seqs])

input_token_index = dict([(token, index) for index, token in enumerate(input_tokens)])
target_token_index = dict([(token, index) for index, token in enumerate(target_tokens)])

rev_input_token_index = dict([(index, token) for index, token in enumerate(input_tokens)])
rev_target_token_index = dict([(index, token) for index, token in enumerate(target_tokens)])

encoder_input_data = np.zeros((len(input_seqs), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_seqs), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_seqs), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_seq, target_seq) in enumerate(zip(input_seqs, target_seqs)):
    for j, token in enumerate(input_seq):
        encoder_input_data[i, j, input_token_index[token]] = 1.
    for j, token in enumerate(target_seq):
        decoder_input_data[i, j, target_token_index[token]] = 1.
        if j > 0:
            decoder_target_data[i, j-1, target_token_index[token]] = 1.

In [6]:
encoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='encoder_inputs')
encoder_inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='encoder_inputs_length')

decoder_targets = tf.placeholder(shape=(None, None), dtype=tf.int32, name='decoder_targets')
decoder_targets_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='decoder_targets_length')

In [ ]:
saver = tf.train.Saver()
save_path = saver.save(sess, "model/attention-seq2seq.ckpt")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(loss_track)
print('loss {:.4f} after {} examples (batch_size={})'.format(loss_track[-1], len(loss_track)*batch_size, batch_size))